In [ ]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss
from catboost import CatBoostClassifier
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import time
import random
from sklearn.ensemble import IsolationForest
from sklearn.decomposition import PCA
from sklearn.neighbors import LocalOutlierFactor

# Pandas setting to display more dataset rows and columns
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **Data import**

In [ ]:
train = pd.read_csv("/kaggle/input/tabular-playground-series-jun-2021/train.csv", low_memory=False)#, nrows=10000)
test = pd.read_csv("/kaggle/input/tabular-playground-series-jun-2021/test.csv", low_memory=False)
train.info(memory_usage="deep")

In [ ]:
test.info(memory_usage="deep")

# **Data analysis**

In [ ]:
# Colors to be used for plots
colors = ["lightcoral", "sandybrown", "darkorange", "mediumseagreen",
          "lightseagreen", "cornflowerblue", "mediumpurple", "palevioletred",
          "lightskyblue", "sandybrown", "yellowgreen", "indianred",
          "lightsteelblue", "mediumorchid", "deepskyblue"]

## Target distribution

In [ ]:
fig, axs = plt.subplots(ncols=2, nrows=1, figsize=(18, 8), gridspec_kw={'width_ratios': [2, 1]})

bars = axs[0].bar(train["target"].value_counts().sort_index().index,
                  train["target"].value_counts().sort_index().values,
                  color=colors,
                  edgecolor="black")
axs[0].set_title("Target distribution", fontsize=20, pad=15)
axs[0].set_ylabel("Count", fontsize=14, labelpad=15)
axs[0].set_xlabel("Target label", fontsize=14, labelpad=10)
axs[0].bar_label(bars, train["target"].value_counts().sort_index().values,
                 padding=3, fontsize=12)
axs[0].bar_label(bars, [f"{x:2.1f}%" for x in train["target"].value_counts().sort_index().values/2000],
                 padding=-20, fontsize=12)
axs[0].margins(0.025, 0.06)
axs[0].grid(axis="y")

pie = axs[1].pie(train["target"].value_counts(sort=False).sort_index().values,
                 labels=train["target"].value_counts(sort=False).sort_index().index,
                 colors=colors,
                 rotatelabels=True,
                 textprops={"fontsize": 14})
axs[1].axis("equal")
plt.show();

Let's check if these classes are evenly distributed in the dataset.

In [ ]:
df = pd.DataFrame()
df["id"] = train["id"]
df["target"] = train["target"]
df["id"] = pd.cut(df["id"], np.arange(0, 201000, 1000), right=False)
values = df.groupby("id")["target"].value_counts(sort=False).values
classes = ["Class_" + str(x) for x in np.arange(9)]

cols = 3
rows = 3
fig, axs = plt.subplots(ncols=cols, nrows=rows, figsize=(16,rows*4), sharey=False)

plt.subplots_adjust(hspace = 0.35)

i=0
for r in np.arange(0, rows, 1):
    for c in np.arange(0, cols, 1):
        axs[r, c].plot(np.arange(0, 200, 1),
                       [values[x] for x in np.arange(i, 1800, 9)],
                       color=colors[i])
        axs[r, c].set_title(classes[i], fontsize=12, pad=5)
        axs[r, c].set_xticks(np.arange(0, 250, 50))
        axs[r, c].set_xticklabels([str(int(x))+"k" for x in axs[r, c].get_xticks()])
        axs[r, c].set_xlabel("Dataframe id")
        axs[r, c].set_ylabel("Class labels qty per 1k rows")
        axs[r, c].set_ylim(0, 320)
        i+=1
fig.suptitle("Class labels distribution in the train dataset", fontsize=20)
plt.show();

So it looks like class labels are distributed pretty evenly across the dataset.

## Original features value distribution in the train dataset

In [ ]:
def make_data_plots(df, i=0):
    """
    Makes value distribution histogram plots for a given dataframe features
    """
    columns = df.drop(["target", "id"], axis=1).columns.values

    cols = 3
    rows = (len(columns) - i) // cols + 1

    fig, axs = plt.subplots(ncols=cols, nrows=rows, figsize=(16,rows*4), sharey=True)
    plt.subplots_adjust(hspace = 0.2)
    for r in np.arange(0, rows, 1):
        for c in np.arange(0, cols, 1):
            if i >= len(columns):
                axs[r, c].set_visible(False)
            else:
                axs[r, c].hist(df[columns[i]].values, bins = 30,
                               color=random.choice(colors),
                               edgecolor="black")
                axs[r, c].set_title(columns[i], fontsize=12, pad=5)
            i+=1

In [ ]:
make_data_plots(train)

## Original features nonzero value distribution in the train dataset

In [ ]:
def make_nonzero_data_plots(df, i=0):
    """
    Makes nonzero value distribution histogram plots for a given dataframe features
    """    
    
    columns = df.drop(["target", "id"], axis=1).columns.values

    cols = 3
    rows = (len(columns) - i) // cols + 1

    fig, axs = plt.subplots(ncols=cols, nrows=rows, figsize=(16,rows*4), sharey=False)
    plt.subplots_adjust(hspace = 0.2)
    for r in np.arange(0, rows, 1):
        for c in np.arange(0, cols, 1):
            if i >= len(columns):
                axs[r, c].set_visible(False)
            else:
                axs[r, c].hist(df[df[columns[i]] > 0][columns[i]].values, bins = 30,
                               color=random.choice(colors),
                               edgecolor="black")
                axs[r, c].set_title(columns[i], fontsize=12, pad=5)
            i+=1

In [ ]:
make_nonzero_data_plots(train)

## Fraction of nonzero values in the both datasets

In [ ]:
x = -1*np.arange(len(test.drop(["id"], axis=1).columns))

fig, ax = plt.subplots(figsize=(16, 30))
bars = ax.barh(x+0.2, train.drop(["id", "target"], axis=1).astype(bool).sum(axis=0).values / train.shape[0],
               height=0.4, color="cornflowerblue", label="Train dataset", edgecolor="black")
bars2 = ax.barh(x-0.2, test.drop(["id"], axis=1).astype(bool).sum(axis=0).values / test.shape[0],
                height=0.4, color="palevioletred", label="Test dataset", edgecolor="black")
ax.set_title("Fraction of nonzero values in the both datasets", fontsize=30, pad=15)
ax.set_ylabel("Feature names", fontsize=20, labelpad=15)
ax.set_xlabel("Fraction of nonzero values", fontsize=20, labelpad=15)
ax.set_xticks(np.arange(0, 0.8, 0.05))
ax.set_yticks(x)
ax.set_yticklabels(list(test.drop(["id"], axis=1).columns.values))
ax.tick_params(axis="x", labelsize=15)
ax.tick_params(axis="y", labelsize=14)
ax.grid(axis="x")
ax.legend(fontsize=15)
ax2 = ax.secondary_xaxis('top')
ax2.set_xticks(np.arange(0, 0.8, 0.05))
ax2.set_xlabel("Fraction of nonzero values", fontsize=20, labelpad=15)
ax2.tick_params(axis="x", labelsize=15)
plt.margins(0.05, 0.01)

## PCA

In [ ]:
pca = PCA(n_components=2)
X_reduced = pd.DataFrame(pca.fit_transform(train.drop(["id", "target"], axis=1)))

fig, ax = plt.subplots(figsize=(16, 9))
ax.scatter(X_reduced[0], X_reduced[1], c=LabelEncoder().fit_transform(train["target"]), cmap="tab10")
ax.set_title("Scatter plot of the original train set reduced to 2 dimensions", fontsize=20, pad=15)
plt.show();

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(train.drop(["id", "target"], axis=1))

pca = PCA(n_components=2)
X_reduced = pd.DataFrame(pca.fit_transform(X_scaled))

fig, ax = plt.subplots(figsize=(16, 9))
ax.scatter(X_reduced[0], X_reduced[1], c=LabelEncoder().fit_transform(train["target"]), cmap="tab10")
ax.set_title("Scatter plot of the scaled train set reduced to 2 dimensions", fontsize=20, pad=15)
plt.show();

As you can see, the data does not have distinct clusters after reducing dimensions.

## Detecting anomalies with IsolationForest

In [ ]:
# iso_forest = IsolationForest(n_jobs=-1, random_state=42, n_estimators=3000)
# iso_forest.fit(train.drop(["id", "target"], axis=1))
# scores = iso_forest.decision_function(train.drop(["id", "target"], axis=1))
# print(f"Anomalies found in the train dataset: {(scores < 0).sum()}")
# to_drop = train.loc[scores < 0].index

In [ ]:
# # Target distribution in detected anomalies
# train.loc[(scores < 0), "target"].value_counts()

## Detecting anomalies with LocalOutlierFactor

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(train.drop(["id", "target"], axis=1))
lof = LocalOutlierFactor(n_jobs=-1)
lof.fit(X_scaled)
scores = lof.negative_outlier_factor_
score_threshold = -1.93
to_drop = train.loc[scores < score_threshold].index
print(f"Anomalies found in the train dataset: {(scores < score_threshold).sum()}")

In [ ]:
# Target distribution in detected anomalies
train.loc[(scores < score_threshold), "target"].value_counts()

# **Data preparation**

In [ ]:
# Dropping anomaly rows detected with Isolation Forest
train.drop(axis=0, index=set(to_drop), inplace=True)
train.shape

In [ ]:
train.drop(["id", "target"], axis=1).duplicated(keep=False).sum()

There are some duplicates in the dataset. If they have identical feature but different target values it may decrease machine learning performance.

In [ ]:
def delete_duplicates(df):
    """
    Finds duplicates in a given DataFrame and deletes rows with identical features values but different target. 
    """
    
    # Copying duplicate rows in a new dataset and getting their indices
    idx = df.drop(["id", "target"], axis=1).duplicated(keep=False)
    duplicates = df.loc[idx == True].copy()
    features = [x for x in duplicates.columns if "feature" in x]
    idx = duplicates["id"]
    
    # Checking if which rows with equal feature values have different target
    indx_to_drop = []
    for index in idx:
        for row in idx:
            if (row != index) and (row not in indx_to_drop):
                if duplicates.loc[index, features].equals(duplicates.loc[row, features]):
                    if duplicates.loc[index, "target"] != duplicates.loc[row, "target"]:
    #                     print(f"Found duplicates with different targets: {index} - {duplicates.loc[index, 'target']} and {row} - {duplicates.loc[row, 'target']}")
                        indx_to_drop.append(index)
                        indx_to_drop.append(row)
    #                 else:
    #                     print(f"Found duplicates with the same target: {index} and {row}")
    
    # Reporting results
    print(f"There are {len(duplicates['id'])} duplicated rows in the dataset.")
    print(f"{len(set(indx_to_drop))} of them have different target. They will be deleted from the dataset.")
    print(f"The datatframe has {len(df['id'])} rows.")
    df.drop(axis=0, index=set(indx_to_drop), inplace=True)
    print(f"After duplicated deletion there are {len(df['id'])} rows.")
    
    return df

In [ ]:
train = delete_duplicates(train)

In [ ]:
# Target feature encoding
encoder = LabelEncoder()
train["target"] = encoder.fit_transform(train["target"])
train["target"].value_counts(sort=False)

In [ ]:
def transform_dataset(data):
    """
    Adds new custom features and transforms original features into custom categories
    """
    
    # Copying features in a temporary dataset which will be transformed with MinMaxScaler
    df = data[[x for x in data.columns if "feature_" in x]].copy()
    
    # Adding custom features
    data["feature_75"] = df.max(axis=1)
    data["feature_76"] = df.mean(axis=1)
    data["feature_77"] = df.median(axis=1)
    data["feature_78"] = df.nunique(axis=1)
    data["feature_79"] = (df == 0).astype(int).sum(axis=1)
    data["feature_80"] = (df != 0).sum(axis=1)
    data["feature_81"] = (df == 0).astype(int).sum(axis=1) / 75
    data["feature_82"] = (df != 0).sum(axis=1) / 75
    
    
    # Scaling original features and adding new features basing on them
    scaled_df = pd.DataFrame(index = data.index.values, columns = df.columns.values)
    for col in df.columns.values:
        scaler = MinMaxScaler()
        scaled_df[col] = scaler.fit_transform(np.array(df[col]).reshape(-1, 1))
    data["feature_83"] = scaled_df.mean(axis=1)
    data["feature_84"] = (scaled_df == 1).sum(axis=1)
    
    # Cutting original features into custom intevals [0, 1), [1, 15), [15, 30) ...
    intervals = np.insert(np.arange(15, 370, 15), 0, [0, 1])
    intervals_text = pd.cut(train["feature_0"], intervals, right=False).value_counts().sort_index().index.astype("string")
    map_dict = dict(zip(intervals_text, list(np.arange(len(intervals_text)))))
    for i, column in enumerate(data.drop(["id"], axis=1).columns):
        if ((i < 75) and (column!="target")):
            data[column] = pd.cut(data[column], intervals, right=False).astype("string")
            data[column].replace(map_dict, inplace=True)
    
    return data

In [ ]:
# Transforming the train dataset and making value distribution plots for new custom features
train = transform_dataset(train)
make_data_plots(train, i=75)

# **Machine Learning**

In [ ]:
X_train = train.drop(["id", "target"], axis=1)
y_train = train["target"]
X_test = transform_dataset(test).drop("id", axis=1)

CatBoost Hyperparameters were optimized using Optuna (see two cells below which are commented in order to save run time).

In [ ]:
# def train_model_cb(trial, X=X_train, y=y_train):
#     """
#     A function to train a model using different hyperparamerters combinations provided by Optuna. 
#     Log loss of validation data predictions is returned to estimate hyperparameters effectiveness.
#     """
#     cat_features = [x for x in X.columns[:75]]
#     preds = 0
#     split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
#     for train_idx, valid_idx in split.split(X, y):
#         X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
#         y_train, y_valid = y.iloc[train_idx], y.iloc[valid_idx]
        
#         # A set of hyperparameters to optimize by optuna
#         cb_params = {
#                  "iterations": trial.suggest_int('iterations', 1000, 10000),
#                  "learning_rate": trial.suggest_float('learning_rate', 0.001, 1.0),
#                  "depth": trial.suggest_int('depth', 1, 6),
#                  "loss_function": 'MultiClass',
#                  "eval_metric": 'MultiClass',
#                  "leaf_estimation_method": trial.suggest_categorical("leaf_estimation_method", ["Newton", "Gradient"]),#, "Exact"]),
#                  "od_type": "Iter",
#                  "early_stopping_rounds": 500,
#                  "l2_leaf_reg": trial.suggest_float('l2_leaf_reg', 0.00001, 10),
#                  "random_strength": trial.suggest_float('random_strength', 1.0, 2.0),
#                  "bagging_temperature": trial.suggest_float('bagging_temperature', 0.0, 10.0),
#                  "border_count": 254,
#                  "use_best_model": trial.suggest_categorical("use_best_model", [False, True]),
#                  "grow_policy": "SymmetricTree",#trial.suggest_categorical("grow_policy", ["SymmetricTree", "Depthwise", "Lossguide"]),
# #                  "max_leaves": trial.suggest_int('max_leaves', 1, 64),
#                  "task_type": "GPU",
#                     }
            
#         model = CatBoostClassifier(**cb_params)
#         model.fit(
#                     X_train, y_train,
#                     eval_set=(X_valid, y_valid),
#                     verbose=False,
#                     cat_features=cat_features
#                 )

#         oof = model.predict_proba(X_valid)
    
#     return log_loss(y_valid, oof)

In [ ]:
# %%time
# Creating Optuna object and defining its parameters
# study = optuna.create_study(direction='minimize')
# study.optimize(train_model_cb, n_trials = 200)

# Showing optimization results
# print('Number of finished trials:', len(study.trials))
# print('Best trial parameters:', study.best_trial.params)
# print('Best score:', study.best_value)

In [ ]:
def train_with_folds(X, y, X_test, model, splits=10):
    """
    Splits a train dataset into folds to train a given model and make test data predictions 
    """
    cat_features = [x for x in X.columns[:75]]
    skf = StratifiedKFold(n_splits=splits, shuffle=True, random_state=42)
    oof_preds = np.zeros((train.shape[0], y.nunique()))
    preds = 0
    feature_importances = 0
    
    for num, (train_idx, valid_idx) in enumerate(skf.split(X, y)):
        X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
        y_train, y_valid = y.iloc[train_idx], y.iloc[valid_idx]
        
        model.fit(X_train, y_train,
                  eval_set=(X_valid, y_valid),
                  verbose=False,
                  cat_features=cat_features)
        
        oof_preds[valid_idx] = model.predict_proba(X_valid)
        preds += model.predict_proba(X_test) / splits
        print(f"\nFold {num} logloss: {log_loss(y_valid, oof_preds[valid_idx])}")
        feature_importances += model.feature_importances_ / splits
    print(f"Overall logloss: {log_loss(y, oof_preds)}")
    
    return preds, feature_importances

In [ ]:
%%time

# The hyperparameters below were given by Optuna
model = CatBoostClassifier(random_state=42,
                           thread_count=4,
                           verbose=False,
                           iterations=5300,
                           learning_rate=0.004996686623648068,
                           grow_policy="SymmetricTree",
                           loss_function="MultiClass",
                           eval_metric="MultiClass",
                           classes_count=9,
                           od_type="Iter",
                           depth=6,
                           l2_leaf_reg=3.994384171429022,
                           random_strength=1.8493809581160419,
                           bagging_temperature=0.6721279933587145,
                           early_stopping_rounds=400,
                           task_type="GPU")

preds, feature_importances = train_with_folds(X_train, y_train, X_test, model)

# **Feature importances**

In [ ]:
df = pd.DataFrame(columns=["Feature", "Importance"])
df["Feature"] = X_test.columns
df["Importance"] = feature_importances/100
df.loc[:74, "Color"] = "cornflowerblue"
df.loc[75:, "Color"] = "palevioletred"
df.sort_values("Importance", axis=0, ascending=False, inplace=True)

legend_lines = [Line2D([0], [0], color="cornflowerblue", lw=10),
                Line2D([0], [0], color="palevioletred", lw=10)]

x = np.arange(0, len(df["Feature"]))
height = 0.4

fig, ax = plt.subplots(figsize=(16, 30))
bars1 = ax.barh(x, df["Importance"], height=height,
                color=df["Color"], edgecolor="black")
ax.set_title("Feature importances", fontsize=30, pad=15)
ax.set_ylabel("Feature names", fontsize=20, labelpad=15)
ax.set_xlabel("Feature importance", fontsize=20, labelpad=15)
ax.set_yticks(x)
ax.set_yticklabels(df["Feature"], fontsize=15)
# ax.set_xlim(0, 0.25)
# ax.set_xticks(np.arange(0, 0.275, 0.025))
ax.tick_params(axis="x", labelsize=15)
ax.grid(axis="x")
ax2 = ax.secondary_xaxis('top')
ax2.set_xlabel("Feature importance", fontsize=20, labelpad=15)
# ax2.set_xlim(0, 0.25)
# ax2.set_xticks(np.arange(0, 0.275, 0.025))
ax2.tick_params(axis="x", labelsize=15)
ax.legend(legend_lines, ["Original features", "Custom features"], fontsize=15, loc=1, bbox_to_anchor=(0, 0, 1, 0.97))
plt.margins(0.04, 0.01)
plt.gca().invert_yaxis()

# **Predictions submission**

In [ ]:
predictions = pd.DataFrame()
predictions["id"] = test["id"]
predictions = pd.concat([predictions, pd.DataFrame(preds, columns=["Class_" + str(x) for x in np.arange(1, 10, 1)])], axis=1)
predictions.to_csv('submission.csv', index=False, header=predictions.columns)
predictions.head()